In [1]:
import requests
import pandas as pd


In [2]:
# 1. 잘 작동하는 URL 설정
url = 'https://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringList'
key= 'ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=='

In [4]:
ptno='2022A48' # 측정소 코드
ym='2020,2021,2022,2023,2024,2025'

# 2. 주소에 있던 파라미터 그대로 설정
params = {
    'serviceKey': key,
    'pageNo': '1',
    'numOfRows': '3000',
    'resultType': 'json',
    'ptNoList': ptno, # 측정소 코드
    'wmyrList': ym,      # 연도
    'wmodList': '01,02,03,04,05,06,07,08,09,10,11,12'        # 월(01, 02, 03월)
}

# API 설정 (생략된 앞부분 파라미터가 있다고 가정)
try:
    # 3. API 요청
    response = requests.get(url, params=params, verify=True) 
    
    if response.status_code == 200:
        data = response.json()
        
        # 4. 데이터 추출
        items = data.get('getWaterMeasuringList', {}).get('item', [])
        
        if items:
            df = pd.DataFrame(items)
            
            # ROWNO가 있다면 인덱스로 설정
            if 'ROWNO' in df.columns:
                df.set_index('ROWNO', inplace=True)
            
            # 5. 모든 요청 항목을 포함한 이름 변경 맵 (Mapping)
            # 원인 지표들을 먼저 배치하고 클로로필a는 마지막에 추가할 예정
#총량지점명	일자	수온(℃)	수소이온농도(ph)	전기전도도(μS/㎝)	용존산소(㎎/L)	BOD(㎎/L)	COD(㎎/L)	부유물질(㎎/L)	총질소(T-N)(㎎/L)	총인(T-P)(㎎/L)	총유기탄소(TOC)(㎎/L)	유량(㎥/s)

            rename_map = {
                'PT_NM': '총량지점명',
                'WMCYMD': '일자',
                'ITEM_TEMP': '수온(℃)',
                'ITEM_PH': '수소이온농도(ph)',
                'ITEM_EC': '전기전도도(μS/㎝)',
                'ITEM_DOC': '용존산소(㎎/L)',
                'ITEM_BOD': 'BOD(㎎/L)',
                'ITEM_COD': 'COD',
                'ITEM_SS': 'COD(㎎/L)',
                'ITEM_TN': '총질소',
                'ITEM_TP': '총인',
                'ITEM_TOC': 'TOC',
                'ITEM_AMNT': '유량',
                'ITEM_CLOA': '클로로필a'
            }

            # 2. 필요한 컬럼만 추출 (맵에 정의된 순서대로 추출됨)
            # dict.keys() 순서대로 컬럼이 구성되므로 클로로필a가 맨 뒤로 갑니다.
            df_analysis = df[list(rename_map.keys())].copy()
            df_analysis.rename(columns=rename_map, inplace=True)

            # 3. 데이터 타입 숫자형으로 변환 (수치형 데이터 전체)
            num_cols = [
                '수온', 'pH', '전기전도도', '용존산소', 'BOD', 'COD', 
                '부유물질', '총질소', '총인', 'TOC', '유량', '클로로필a'
            ]
            df_analysis[num_cols] = df_analysis[num_cols].apply(pd.to_numeric, errors='coerce')

            # 4. 날짜 데이터 형식 변환
            df_analysis['측정일자'] = pd.to_datetime(df_analysis['측정일자'], errors='coerce')

            # 5. 결과 확인
            print("--- [데이터 정제 완료: 클로로필a 타겟 배치] ---")
            pd.set_option('display.max_columns', None)  # 전체 컬럼 출력 설정
            pd.set_option('display.width', 1000)
            print(df_analysis.head())
            
            # (옵션) 마지막 열이 클로로필a인지 다시 한 번 확인
            print(f"\n마지막 열 확인: {df_analysis.columns[-1]}")
            
        else:
            print("응답은 성공했으나 데이터가 없습니다.")
    else:
        print(f"API 요청 실패: {response.status_code}")

except Exception as e:
    print(f"오류 발생: {e}")

응답은 성공했으나 데이터가 없습니다.


In [ ]:
df = pd.read_csv('data/nakdong_water_quality.csv',encoding='euc-kr')
df[df['총량지점명']=='낙본M']='하구원1'

0      낙본M
1      낙본N
2      낙본M
3      낙본N
4      낙본M
      ... 
363    낙본N
364    낙본M
365    낙본N
366    낙본M
367    낙본N
Name: 총량지점명, Length: 368, dtype: object

In [ ]:
df


In [10]:
df_analysis

,지점명,측정일자,수온,pH,전기전도도,용존산소,BOD,COD,부유물질,총질소,총인,TOC,유량,클로로필a
ROWNO,,,,,,,,,,,,,,
1,금곡,2020-01-20,5.6,8.2,351.0,14.7,2.0,5.6,4.4,3.280,0.031,4.7,131.43,29.9
2,금곡,2020-01-14,6.1,8.1,334.0,16.5,1.7,5.0,5.0,3.189,0.026,4.2,248.26,16.2
3,금곡,2020-01-29,7.4,8.2,352.0,13.9,2.0,4.8,5.4,3.240,0.030,4.2,508.79,21.0
4,금곡,2020-02-03,6.9,8.7,330.0,15.1,2.6,5.9,8.0,3.368,0.036,4.5,145.60,44.0
5,금곡,2020-02-25,8.6,8.1,339.0,12.6,2.6,5.4,8.0,3.405,0.033,5.2,311.48,30.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,신어천3,2025-06-10,28.2,9.5,364.0,11.6,9.1,15.2,19.6,1.656,0.127,5.1,NaN,69.5
1733,신어천3,2025-07-10,30.0,7.7,424.0,8.5,3.9,6.5,24.9,1.083,0.135,5.4,NaN,75.8
1734,신어천3,2025-08-07,29.3,7.7,293.0,6.0,1.6,4.5,5.9,1.574,0.099,2.4,NaN,0.7
